In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import os
from owi_data_2_pandas.io import API
import datetime
from pytz import utc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.dates as mdates

# sklearn imports
from sklearn.model_selection import train_test_split

# oma_tracking imports
from oma_tracking.data.preprocessing import sin_cos_angle_inputs

# Import vub Meetenet Vlaamse Banken API functions
from vubmvbc.config import Credentials
from vubmvbc.client import Base
from vubmvbc.objects import Catalog, Data
import vubmvbc.data_getter as dg

from oma_tracking.oma_clustering import ModeClusterer  #pip install hdbscan --no-build-isolation --no-binary :all
from oma_tracking.data.make_dataset import DatasetGetter
from oma_tracking.data.utils import unpack_mode, get_frequencies, get_rated_data, get_parked_data, read_simulations_csv_files

In [51]:
start  = datetime.datetime(2012,1,1,tzinfo=utc)
stop  = datetime.datetime(2023,7,19,tzinfo=utc)

location = 'bbc01'
name_location = 'BB_C01'

home_folder = "../../"
data_file_name = '_'.join([location, start.strftime("%Y%m%d"), stop.strftime("%Y%m%d")])
data_path = home_folder + "data/bb/raw/" + data_file_name + ".parquet"
data = pd.read_parquet(data_path)


SS1 = get_frequencies(data, 'SS1')
SS2 = get_frequencies(data, 'SS2')
FA1 = get_frequencies(data, 'FA1')

In [85]:
weather_data_path = "../../data/nw2/mvbc_data_longterm.parquet"
Thorntonbank_data_path = "../../data/nw2/Thorntonbank_data_longterm.parquet"
Westhinder_data_path = "../../data/nw2/Westhinder_data_longterm.parquet"

weather_data = pd.read_parquet(weather_data_path)

scada_inputs = data.filter(regex='mean')

inputs = \
    pd.concat(
        [
            weather_data,
            scada_inputs
        ],
        axis=1
    )

selected_columns = \
    [
       'mvbc_WandelaarBuoy_10%_highest_waves',
       'mvbc_WandelaarBuoy_Height_waves_with_period_>_10_s',
       'mvbc_WandelaarMeasuringpile_Flow_direction_cell_3',
       #'mvbc_WandelaarBuoy_Wave_height',
       #'mvbc_WandelaarBuoy_Average_wave_period',
       #'mvbc_WandelaarMeasuringpile_Tide_TAW',
       #'mvbc_WandelaarMeasuringpile_Air_pressure',
       #'mvbc_WandelaarMeasuringpile_Air_temperature',
       #'mvbc_Wandelaar_Sea_water_temperature',
       'Sea_water_temperature',
       'Wave_height',
       'Tide_TAW',
       'Average_wave_period',
       'Air_pressure',
       'Air_temperature',
       'mean_'+ name_location + '_rpm',
       'mean_'+ name_location + '_yaw',
       'mean_'+ name_location + '_pitch',
       'mean_'+ name_location + '_power',
       'mean_'+ name_location + '_windspeed',
       'mean_'+ name_location + '_winddirection'
    ]
#inputs = inputs[selected_columns]

In [88]:
1 - scada_inputs.isna().sum()/len(scada_inputs)

mean_BB_C01_rpm              0.989941
mean_BB_C01_yaw              0.989939
mean_BB_C01_pitch            0.989941
mean_BB_C01_windspeed        0.989919
mean_BB_C01_winddirection    0.989941
mean_BB_C01_power            0.984614
dtype: float64

In [86]:
1 - inputs.isna().sum()/len(inputs)

mvbc_WandelaarBuoy_10%_highest_waves                                    0.948774
mvbc_WandelaarBuoy_Wave_height                                          0.948774
mvbc_WandelaarBuoy_Average_wave_period                                  0.947462
mvbc_WandelaarBuoy_Height_waves_with_period_>_10_s                      0.929785
mvbc_WandelaarMeasuringpile_Sea_water_temperature                       0.519111
mvbc_WandelaarMeasuringpile_Max_3-seconds_wind_gust_(at_10_m_height)    0.797223
mvbc_WandelaarMeasuringpile_Average_wind_direction                      0.932549
mvbc_WandelaarMeasuringpile_Tide_TAW                                    0.930750
mvbc_WandelaarMeasuringpile_Air_pressure                                0.917975
mvbc_WandelaarMeasuringpile_Flow_direction_cell_3                       0.430598
mvbc_WandelaarBuoy_1%_wave_height                                       0.494059
mvbc_WandelaarMeasuringpile_Average_wind_speed_(at_10_m_height)         0.797156
mvbc_WandelaarMeasuringpile_

In [91]:
1 - SS1.isna().sum()/len(SS1)

mean_frequency    1.0
dtype: float64

In [71]:
### RECURSIVE FEATURE ELIMINATION (RFE) ###
import xgboost as xgb
from shaphypetune import BoostSearch, BoostRFE, BoostRFA, BoostBoruta
import warnings
warnings.filterwarnings('ignore')

y = SS1.copy().dropna()
X = inputs.copy()
X = X[X.columns[(X.isna().sum() < len(X)/2.9)]]
X = X.loc[y.index].dropna()
y = y.loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

regr_xgb = xgb.XGBRegressor()
model = BoostRFE(regr_xgb, step=1, importance_type='shap_importances',
            train_importance=True, min_features_to_select=5, verbose=2)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=2)

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
print(r2_score(y_test, model.predict(X_test)))
print(mean_squared_error(y_test, model.predict(X_test)))
print(mean_absolute_error(y_test, model.predict(X_test)))
print(mean_absolute_percentage_error(y_test, model.predict(X_test)))

print('kept columns', X.columns[model.support_])
print('removed columns', X.columns[~model.support_])

Fitting estimator with 14 features
Fitting estimator with 13 features
Fitting estimator with 12 features
Fitting estimator with 11 features
Fitting estimator with 10 features
Fitting estimator with 9 features
Fitting estimator with 8 features
Fitting estimator with 7 features
Fitting estimator with 6 features
Fitting estimator with 5 features
0.30472766210301716
6.9872174335816675e-06
0.002032001924523809
0.005517353508892736
kept columns Index(['mvbc_WandelaarBuoy_10%_highest_waves',
       'mvbc_WandelaarBuoy_Height_waves_with_period_>_10_s',
       'Sea_water_temperature', 'Wave_height', 'Tide_TAW',
       'Average_wave_period', 'Air_pressure', 'Air_temperature',
       'mean_BB_C01_rpm', 'mean_BB_C01_yaw', 'mean_BB_C01_pitch',
       'mean_BB_C01_power', 'mean_BB_C01_windspeed',
       'mean_BB_C01_winddirection'],
      dtype='object')
removed columns Index([], dtype='object')


## SS2 data selection

In [72]:
parked_data = get_parked_data(data)
parked_SS2 = get_frequencies(parked_data, 'SS2')
days = parked_SS2.groupby(parked_SS2.index.date).count()[parked_SS2.groupby(parked_SS2.index.date).count() > 144*0.2].dropna().index
parked_SS2['date'] = parked_SS2.index.date
parked_SS2_longterm = parked_SS2[np.isin(parked_SS2['date'], days)].drop(columns=['date'])

from sklearn.cluster import DBSCAN

time_difference = (parked_SS2_longterm.index-parked_SS2_longterm.index[0]).total_seconds()/60
parked_SS2_longterm['time_difference'] = time_difference
clustering = DBSCAN(eps=60*24*14, min_samples=2).fit(parked_SS2_longterm['time_difference'].values.reshape(-1, 1) )
parked_SS2_longterm['cluster'] = clustering.labels_

big_clusters = parked_SS2_longterm.groupby('cluster').count()[parked_SS2_longterm.groupby('cluster').count()>144].dropna().index
parked_SS2_longterm_big_clusters = parked_SS2_longterm[np.isin(parked_SS2_longterm['cluster'], big_clusters)]
unique_vals = parked_SS2_longterm_big_clusters['cluster'].unique()
parked_SS2_longterm_big_unique_clusters = parked_SS2_longterm_big_clusters['cluster'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))),
           inplace=True)

In [78]:
### RECURSIVE FEATURE ELIMINATION (RFE) ###
import xgboost as xgb
from shaphypetune import BoostSearch, BoostRFE, BoostRFA, BoostBoruta
import warnings
warnings.filterwarnings('ignore')

y = parked_SS2_longterm['mean_frequency'].copy().dropna()
X = inputs.copy()
X = X[X.columns[(X.isna().sum() < len(X)/2.9)]]
X = X.loc[y.index].dropna()
y = y.loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

regr_xgb = xgb.XGBRegressor()
model = BoostRFE(regr_xgb, step=1, importance_type='shap_importances',
            train_importance=True, min_features_to_select=5, verbose=2)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=2)

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
print(r2_score(y_test, model.predict(X_test)))
print(mean_squared_error(y_test, model.predict(X_test)))
print(mean_absolute_error(y_test, model.predict(X_test)))
print(mean_absolute_percentage_error(y_test, model.predict(X_test)))

print('kept columns', X.columns[model.support_])
print('removed columns', X.columns[~model.support_])

Fitting estimator with 14 features
Fitting estimator with 13 features
Fitting estimator with 12 features
Fitting estimator with 11 features
Fitting estimator with 10 features
Fitting estimator with 9 features
Fitting estimator with 8 features
Fitting estimator with 7 features
Fitting estimator with 6 features
Fitting estimator with 5 features
0.872466010603602
0.00010351433582788975
0.006609062186785045
0.0044591928460253035
kept columns Index(['mvbc_WandelaarBuoy_Height_waves_with_period_>_10_s',
       'Sea_water_temperature', 'Wave_height', 'Tide_TAW', 'Air_pressure',
       'Air_temperature', 'mean_BB_C01_rpm', 'mean_BB_C01_yaw',
       'mean_BB_C01_pitch', 'mean_BB_C01_power', 'mean_BB_C01_windspeed',
       'mean_BB_C01_winddirection'],
      dtype='object')
removed columns Index(['mvbc_WandelaarBuoy_10%_highest_waves', 'Average_wave_period'], dtype='object')


In [95]:
1 - len(parked_SS2_longterm)/len(inputs)

0.9671835586846421

In [101]:
len(SS1)/len(inputs)

0.8493499608132405

In [100]:
len(parked_SS2_longterm)/len(inputs)

0.032816441315357915